# Covid 19 Data Visualization 

Dünyanın ve ülkelerin zamana bağlı değişim grafikleri ve kıtalara göre ülkelerdeki vaka, ölüm verilerini görselleştirdim.

Verileri çektiğim api'da güncel covid19 verileri, regions.csv dosyasında ise ülkelerin bulunduğu kıtaların listesi mevcut. Bu iki veri setini birleştirip ülkelerin güncel covid19 durumunu kıtalara göre grupladım. 

Grafikler inline olduğu için detayı gözükmüyor, jupyterde açıp %matplotlib qt komutunu girerseniz farklı bir pencerede açılır, yaklaştırıp uzaklaştırabilirsiniz.


In [ ]:
import pandas as pd
import json
import requests
from pandas import json_normalize
from matplotlib import pyplot as plt


### Api verileri çok karmaşık sunuyor, manuel olarak parse etmek saçma olacağı için json_normalize methoduyla parse ettim

In [ ]:
response = requests.get('https://corona-api.com/countries').json()['data']

### Api'de 248 ülkeye ait:
* Vaka Sayıları
* Ölüm Sayıları
* İyileşen Sayıları
* Timeline verileri mevcut, işimize yaramayacak başka verilerde var, az sonra temizleyeceğim.
 

In [ ]:
covid = json_normalize(response)

In [ ]:
covid.shape

In [ ]:
covid.head(10)

latest_data -> Toplam veriyi ifade ediyor

### Gereksiz bilgileri silebiliriz

Elimizde ülkelerin günlük ve toplam covid19 verileri gibi gerekli bilgiler olduğu gibi, koordinat ve güncelleme gibi gereksiz bilgilerde var.Bunlardan kurtulmak istiyorum.

In [ ]:
covid.drop(['updated_at','coordinates.latitude','coordinates.longitude','latest_data.calculated.recovered_vs_death_ratio',],
           axis = 1, inplace=True)

In [ ]:
covid.head()

### Kıtaları çekiyorum

Ülkelerin bulunduğu kıtaları regions.csv dosyasından çekiyorum

In [ ]:
data = pd.read_csv('../input/country-mapping-iso-continent-region/continents2.csv')

In [ ]:
regions = pd.DataFrame(data)

In [ ]:
regions.head()

Bu veri setinde ihtiyaç duyduğum tek şey ülkelerin alpha2 kodu ve bulundukları kıtalar, işime yaramayan sütunlardan kurtuluyorum.

In [ ]:
cols_to_keep = ['alpha-2','region']

In [ ]:
regions = regions[cols_to_keep]

In [ ]:
regions.head()

### Covid verileriyle kıta verilerini birleştiriyorum

Merge işlemini ülkelerin ISO kodlarına göre yapacağım.Covid verilerinin bulunduğu DataFrame'de ISO kodlarını tuttuğum sütunun ismi code , regions adlı DataFrame'de ise alpha-2. Merge işlemini yapabilmek için alpha-2 adlı sütunun ismini code yapıyorum. 

In [ ]:
regions.rename(columns = {'alpha-2':'code'}, inplace = True)

In [ ]:
data = covid.merge(regions , on='code')

### Ülkeleri vaka sayısına göre sıralıyorum

In [ ]:
data.sort_values(by="latest_data.confirmed", ascending=False, inplace=True)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
del data['index']

In [ ]:
data

### Ölüm oranı NaN olan ülkelerde vaka yok, bu ülkeleri silebiliriz

In [ ]:
data.dropna(subset=['latest_data.calculated.death_rate'],inplace=True)

In [ ]:
data.shape

Ülke sayısı 248'den 205'e düştü

### Ülkeleri kıtalara göre grupluyorum

In [ ]:
data["region"].value_counts()

In [ ]:
grouped_data = data.groupby(["region"])

In [ ]:
grouped_data.get_group("Asia")

Asya kıtasındaki ülkeleri toplam vaka sayısına göre sıralıyorum

In [ ]:
continent = grouped_data.get_group("Asia").sort_values(by="latest_data.confirmed", ascending=False)

Yukarıda ülkeleri vaka sayısına göre sıraladığımız için şimdi kıtalara göre grupladığımız halde, hangi ülkenin dünyada kaçıncı sırada olduğunu görebiliyoruz

In [ ]:
continent.head()

# Dünyanın ve Ülkelerin Zamana Bağlı Değişimleri

### İstekte bulunduğum api verileri ülkelerin alpha2 koduna göre sunuyor. Bunun için ülke adını alpha2 koduna çevirecek bir fonksiyon tanımlıyorum.

In [ ]:
def alpha_2(country_name):
        with open('../input/alpha2/ulkeler.json' ,encoding="utf8") as f:
            data = json.load(f)
        for alpha_code,name in data.items():
            if name.lower() == country_name.lower():
                code = alpha_code
                return code
        if country_name[0] == "i" or country_name[0] == "I" :
            raise NameError(f'{country_name} ülkesinin ilk harfini şu şekilde yazın => {"İ"+country_name[1:]}')
        else:
            raise NameError(f'{country_name} değerine karşılık gelen bir alpha-2 kodu bulunamadı')

### Ülkelerin ve Dünyanın timeline grafiğini çizecek fonksiyonu tanımlıyorum.
Hiç parametre verilmezse dünyanın grafiğini çizecek. Parametre olarak herhangi bir ülke adı verilirse o ülkenin grafiğini çizecek


In [ ]:
def timeline_graph(arg="World"):
        if arg == 'World':
            response = requests.get('https://corona-api.com/timeline').json()['data']
        else:
            code = alpha_2(arg)
            response = timeline = requests.get(f'https://corona-api.com/countries/{code}').json()['data']['timeline']
            
        timeline = response
        data = {'dates':[],'confirmed':[],'deaths':[],'recovered':[]}
        for day in timeline:
            data['dates'].append(day['date'])
            data['confirmed'].append(day['confirmed'])
            data['deaths'].append(day['deaths'])
            data['recovered'].append(day['recovered'])
                  
   
        plt.rcParams["figure.figsize"] = [16,9]
        plt.style.use("fast")
        plt.plot(data.get('dates')[::-1],data.get('confirmed')[::-1],  label = "confirmed")
        plt.plot(data.get('dates')[::-1],data.get('deaths')[::-1],  label = "deaths")
        plt.plot(data.get('dates')[::-1],data.get('recovered')[::-1],  label = "recovered")


        
        plt.xlabel("Tarih")
        
        plt.xticks(rotation='vertical', fontsize='8')
        plt.ylabel("Sayı")
        plt.tight_layout()
        plt.title(f"Covid19 {arg} Timeline")

        plt.legend()

        plt.show()

    

### Fonksiyona parametre vermezsek tüm dünyanın zamana bağlı değişim grafiğini çizecek

In [ ]:
timeline_graph()

### Artık 205 ülkeden herhangi birini parametre olarak verip grafiğini çizdirebiliriz 

In [ ]:
timeline_graph("Türkiye")

In [ ]:
timeline_graph("Amerika")

Uzatmamak adına Türkiye ve Amerikayı çizdirdim, fakat aynı şekilde diğer ülkelerin de grafiğini çizdirebiliriz 

# Kıta Bazında Görselleştirme

Şimdi bir fonksiyon tanımlayacağım, parametre olarak verilen kıtadaki ülkelerin vaka sayısı grafiğini çizecek.Program notebook üzerinde çalıştığı için grafikler küçük gözüküyor.
%matplotlib qt komutunu çalıştırıp grafikleri ayrı bir sekmede açabilirsiniz

In [ ]:
def confirmed_graph(kita):
    plt.rcParams["figure.figsize"] = [16,9]
    plt.style.use("seaborn-ticks")
    continent = grouped_data.get_group(kita).sort_values(by="latest_data.confirmed", ascending=False)

    plt.barh(continent.iloc[0:15]['name'], continent.iloc[0:15]['latest_data.confirmed'],  label = "# of case")

    plt.xlabel("Cases")
    plt.ylabel("Country")
    plt.title(f"Covid19 {kita} verileri")

    plt.legend(loc='upper right')
    plt.grid(True)

    plt.tight_layout()

    plt.show() 
    

### Bu fonksiyon verdiğiniz kıtadaki en fazla vaka olan 15 ülkeyi barplot grafiğine döker

In [ ]:
confirmed_graph("Asia")

In [ ]:
confirmed_graph("Europe")

### İşi biraz daha ileriye götürüp fonksiyonu şöyle değiştirebiriz.

In [ ]:
def graph(kita,data,log=False):
    plt.rcParams["figure.figsize"] = [16,9]
    plt.style.use("seaborn-ticks")
    continent = grouped_data.get_group(kita).sort_values(by=f"latest_data.{data}", ascending=False)

    plt.barh(continent.iloc[0:15]['name'], continent.iloc[0:15][f'latest_data.{data}'],  label = f"# of {data}", log=log)

    plt.xlabel("Cases")
    plt.ylabel("Country")
    plt.title(f"{kita} | {data}")

    plt.legend(loc='upper right')
    plt.grid(True)

    plt.tight_layout()

    plt.show() 
    

### Artık fonksiyona 3 parametre veriyoruz,Kıtanın ismi ,hangi veriyi istediğimiz ve artışın logaritmik olup olmayacağı.Verdiğimiz parametreye göre grafiği çizecek(deaths,confirmed vs.)
Örneğin Amerika kıtasında ülkeler arasındaki fark Amerika Birleşik Devletlerinde vaka sayının diğerlerine göre çok fazla olmasından dolayı artışın logaritmik olması daha mantıklı olacaktır. Aşağıda bununla alakalı bir örnek yaptım. 

### Asya kıtasında en fazla ölüm olan 15 ülkenin grafiği 

In [ ]:
graph("Asia","deaths")

### Avrupa kıtasında en çok vaka sayısı bulunan 15 ülkenin grafiği

In [ ]:
graph("Europe",'confirmed')

### Amerika kıtasında en çok ölüm olan 15 ülkenin grafiği

In [ ]:
graph("Americas","deaths")

**Cuba gibi ülkelerde vaka olmasına rağmen Amerika Birleşik Devletlerindeki vaka sayısının fazlalığından dolayı grafikte gözükmüyor. Artışı logaritmik yapıyorum: **

In [ ]:
graph("Americas","deaths",log=True)

### Aşağıda tanımladığım piechart( ) fonksiyonu ise parametre olarak verilen kıtanın, yine parametre olarak verilen ülkenin verisine göre pasta grafiğini çiziyor

In [ ]:
def piechart(kita,data):
    continent = grouped_data.get_group(kita).sort_values(by=f"latest_data.{data}", ascending=False)
    plt.rcParams["figure.figsize"] = [25,16]
    plt.style.use("fast")
    

    plt.title(f"{kita} | {data}")
    

    plt.pie(continent.iloc[0:10][f'latest_data.{data}'], labels =continent.iloc[0:10]['name'] , shadow=True,
            startangle=70, autopct="%2.1f%%", 
            wedgeprops={"edgecolor":"black"})

    plt.tight_layout()

    plt.show( )

### Asya kıtasındaki ülkelerin vaka sayısı oranları

In [ ]:
piechart("Asia","confirmed")

### Avrupa kıtasındaki ülkelerin ölüm sayısı oranları

In [ ]:
piechart("Europe","deaths")